In [8]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import copy
from IPython.display import display, HTML

In [3]:
# Preprocessing.
# Conversion all the stuff to the numeric format.
print('Loading...')
main_data = pd.read_excel('Dataset.xlsx')
parameters_definition = pd.read_excel('Описание параметров.xlsx')
print('Converting...')
for i, c in enumerate(main_data.columns):
    if main_data[c].dtype == 'datetime64[ns]':
        main_data[c] = main_data[c].values.astype(np.int64) // 1e9
    if main_data[c].dtype == 'object':
        main_data[c] = [ float(re.sub('[.\d]', '',                      # This creepy regexp
                                      re.sub('[^\d\.]', '',             # removes alphabetic symbols
                                             re.sub(',', '.', item)),   # replaces ',' with '.'
                                      1                                 # removes '.' before the first digit
                                      )                                 # and needs to be refactored.
                               )
                         for item
                         in main_data[c].values
                       ]
print('Data converted!')

Loading...
Converting...
Data converted!


In [4]:
# Checking the data for NaNs, zeros and ranges.
# Cleaning the data:
# removing parameters with big number of broken values;
# removing parameters with the same values;
# manual removing of broken time-related columns "поступление в тех.секцию" and "выезд из тех.секции";
# manual removing of all cold rolling lengths (zero lengths and strange values)
# manual removing of post-parameters

removed_parameters = ['Поступление_в_технологическую_секцию',
                      'Выезд_из_технологической_секции',
                      'Начало непрерывного отжига',
                      'Окончание непрерывного отжига',
                      'Длина_1',
                      'Длина_2',
                      'Длина_3',
                      'Длина_4',
                      'Длина_5',
                      'Коэффициент_свойств_стали',
                      'Средние_магнитные_потери']

print('Checking the data for NaNs and similar values:\n')

UNIQUE_CUT = 1

for c in main_data.columns:
    if not c in removed_parameters:
        broken_flag = False
        inds = main_data[c].index[main_data[c].apply(np.isnan)]
        unique = np.unique(main_data[c].values)
        valmin = np.nanmin(main_data[c].values)
        valmax = np.nanmax(main_data[c].values)
        if len(inds) > 0:
            print('! \t'+c+'\tDType '+str(main_data[c].dtype)+'\tRange = ['+str(valmin)+' '+str(valmax)+'].\t Contains '+str(len(inds))+' NaNs at: '+str(inds.values))
            broken_flag = True
        if len(unique) <= UNIQUE_CUT:
            print('! \t'+c+'\tDType '+str(main_data[c].dtype)+'\tCointains similar values '+str(unique))
            broken_flag = True
        else:
            #        print('OK\t'+c+'\tDType '+str(main_data[c].dtype)+'\tRange = ['+str(valin)+' '+str(valmax)+'].\t Len(unique): '+str(len(np.unique(main_data[c].values))))
            pass
        if broken_flag:
            removed_parameters.append(c)
print('\n'+str(len(removed_parameters))+' parameters are removed as non-informative:')
print(removed_parameters)
valid_parameters = list( set(main_data.columns) - set(removed_parameters) )
#print('Valid parameters:')
#print(valid_parameters)
clean_data = main_data[valid_parameters].copy()

print('\nClean dataframe saved.')

Checking the data for NaNs and similar values:

! 	Cu	DType float64	Cointains similar values [0.01]
! 	Ni	DType float64	Cointains similar values [0.01]
! 	H2_Этап3_зона3	DType float64	Cointains similar values [0.10204082]
! 	T_Этап1_зона6	DType int64	Cointains similar values [0]
! 	Точка_росы_Этап1_зона1	DType float64	Range = [62.4153658537 63.91890625].	 Contains 7120 NaNs at: [    0     1     2 ... 12788 12789 12790]
! 	Точка_росы_Этап1_зона2	DType float64	Range = [61.91 64.0462].	 Contains 5671 NaNs at: [  739   740   741 ... 11459 11460 11461]
! 	CO_Этап1_зона1	DType float64	Range = [1.49 1.63].	 Contains 11476 NaNs at: [    0     1     2 ... 12788 12789 12790]
! 	CO_Этап1_зона2	DType float64	Range = [0.0332926829 1.6529333333].	 Contains 1315 NaNs at: [10046 10047 10048 ... 12078 12079 12080]

19 parameters are removed as non-informative:
['Поступление_в_технологическую_секцию', 'Выезд_из_технологической_секции', 'Начало непрерывного отжига', 'Окончание непрерывного отжига', 'Длин

In [45]:
# Runs linear regression model for the estimation of the influence of given parameters to specific magnetic loss.
# Prints a tables with results on importance of parameters and production steps.

def steps_x_pars(pars, definitions):
    '''Function for association the steps with the parameters.
    TODO: May be written more elegant way.'''
    steps = []
    for p in pars:
        for i, d in enumerate(definitions['Название параметра']):
            if d==p:
                steps.append(definitions['Этап обработки'][i])
    return steps

def calculate_mean_importance(production_steps_unique, important_pars):
    '''Calculates mean and median Rsq for steps of production.'''
    mean_rsq = []
    median_rsq = []
    for step in production_steps_unique:
        list_of_rsq_for_current_step = []
        for p in important_pars:
            if p[1]==step:
                list_of_rsq_for_current_step.append(p[2])
        mean_rsq.append(np.mean(list_of_rsq_for_current_step))
        median_rsq.append(np.median(list_of_rsq_for_current_step))
    return mean_rsq, median_rsq

r_sq_list = []
model_list = []
SML = np.array(clean_data['Удельные_потери'])
TOP_CUT = 25 # number of important parameters for print

for c in clean_data.columns:
    data = np.array(clean_data[c].values).reshape((-1,1))
    model = LinearRegression().fit(data, SML)
    r_sq = model.score(data, SML)
    r_sq_list.append(r_sq)
    model_list.append(copy.deepcopy(model))

production_steps =  steps_x_pars(clean_data.columns, parameters_definition)
important_pars = sorted(zip(clean_data.columns, production_steps, r_sq_list, model_list), key=lambda x: x[2], reverse=True)

print(str(TOP_CUT)+' наиболее важных параметров: ')
display(HTML((pd.DataFrame(np.array(important_pars)[1:TOP_CUT+1,:3], columns = ['Параметр', 'Этап', 'Важность (модельное СКО)']).to_html()))) # 1st entry contains SML

production_steps_unique = np.unique(production_steps)
mean_rsq, median_rsq = calculate_mean_importance(production_steps_unique, important_pars)
zipped_steps = sorted(zip(production_steps_unique, mean_rsq, median_rsq), key=lambda x: x[1], reverse= True)

print('\nВажность отдельных этапов:')
display(HTML(pd.DataFrame(zipped_steps[1:], columns = ['Этап', 'СКО (среднее)', 'СКО (медианное)']).to_html())) # 1st entry contains post-production step

25 наиболее важных параметров: 


,Параметр,Этап,Важность (модельное СКО)
0,N,Химия плавки,0.257631
1,F,Химия плавки,0.238945
2,Al,Химия плавки,0.223502
3,ШОС,Химия плавки,0.152245
4,Cr,Химия плавки,0.149097
5,T_проход_3,Холодный прокат,0.14155
6,Азот_среднее,Химия итог,0.137227
7,S,Химия плавки,0.134812
8,Номер плавки,Общие данные,0.129961
9,T_проход_1,Холодный прокат,0.108705



Важность отдельных этапов:


,Этап,СКО (среднее),СКО (медианное)
0,Химия плавки,0.125137,0.134812
1,Холодный прокат,0.061048,0.031737
2,Общие данные,0.051044,0.040759
3,Химия итог,0.032932,0.017972
4,Горячий прокат,0.023967,0.004909
5,Обезуглероживание,0.016798,0.003606
6,Нормализация,0.002551,0.000138


In [ ]:
# Not necessary. Just for show the way of thinking.
# Plotting the fits of top important parameters.
# TOP_CUT constant is defined in the previous block
def par_vs_sml(par, model):
    ''' Plots parameter-on-parameter direct distribution and model fit.'''
    plt.plot(clean_data['Удельные_потери'].values, clean_data[par].values, 'x')
    plt.plot(model.predict(np.array(clean_data[par].values).reshape((-1,1)) ), clean_data[par].values,  label='Model')
    #    print(model.predict(np.array(clean_data['Удельные_потери'].values).reshape((-1,1))))
    plt.title(par)
    plt.ticklabel_format(useOffset=False)
    plt.xlabel('Удельные магнитные потери')
    plt.ylabel(par)
    plt.show()
    plt.clf()
for entry in important_pars[1:TOP_CUT+1]:
    par_vs_sml(entry[0],entry[3])

In [ ]:
# Not necessary. Just for show the way of thinking.
# Generates a lot of plots (dependence of time on melting and brigade numbers).
# By this I found that times are connected with number of melting, so it can be reconstructed by this.

def par_vs_par(par1, par2, only_abs = True):
    ''' Plots parameter-on-parameter direct distribution.
    only_abs argument enables abs-value cut for both parameters by the value of PARAMETER 1 (Y-axis) '''
    if only_abs:
        plt.plot(main_data[par2].values[tuple(np.where(main_data[par1].values > 0))],
                 main_data[par1].values[tuple(np.where(main_data[par1].values > 0))], 'x')
        plt.title('Only abs cut of '+par2)
    else:
        plt.plot(main_data[par2].values[tuple(np.where(main_data[par1].values != np.nan))],
                 main_data[par1].values[tuple(np.where(main_data[par1].values != np.nan))], 'x')
        plt.title('All values of '+par2)
    plt.ticklabel_format(useOffset=False)
    plt.xlabel(par2)
    plt.ylabel(par1)
    plt.show()
    plt.clf()
    return 0
def real_time_histogram(time, parameter):
    ''' Takes names of checked parameters as 'strings'
    Plots the distribution. '''
    cut_indexes = (np.where(main_data[time] > 0))
    parameter_cut = main_data[parameter].values[tuple(cut_indexes)]
    plt.hist(parameter_cut)
    plt.title(time + ' (правильное)')
    plt.xlabel(parameter)
    plt.ylabel('N')
    plt.show()
    plt.clf()
def dead_time_histogram(time, parameter):
    ''' Takes names of checked parameters as 'strings'
    Plots the distribution. '''
    cut_indexes = (np.where(main_data[time] < 0))
    parameter_cut = main_data[parameter].values[tuple(cut_indexes)]
    plt.hist(parameter_cut)
    plt.title(time + ' (сломанное)')
    plt.xlabel(parameter)
    plt.ylabel('N')
    plt.show()
    plt.clf()

print('Earliest: '+str(pd.to_datetime((np.min(main_data['Выезд_из_технологической_секции'])) * 1e9)))
print('Latest: '  +str(pd.to_datetime((np.max(main_data['Выезд_из_технологической_секции'])) * 1e9)))

# Lets check the times and numbers...
print('Поступление_в_технологическую_секцию '+str(np.unique(main_data['Поступление_в_технологическую_секцию'].values)))
print('Выезд_из_технологической_секции '+str(np.unique(main_data['Выезд_из_технологической_секции'].values)))
print('Номер плавки '+str(np.unique(main_data['Номер плавки'].values)))

# Hmm, what about the distribution of broken times on the melting numbers?
par_vs_par('Поступление_в_технологическую_секцию', 'Номер плавки')
par_vs_par('Поступление_в_технологическую_секцию', 'Номер плавки', only_abs = False)
par_vs_par('Выезд_из_технологической_секции', 'Номер_бригады')
par_vs_par('Выезд_из_технологической_секции', 'Номер_бригады', only_abs = False)

# Histograms of times...
real_time_histogram('Поступление_в_технологическую_секцию', 'Номер плавки')
dead_time_histogram('Поступление_в_технологическую_секцию', 'Номер плавки')
real_time_histogram('Выезд_из_технологической_секции', 'Номер плавки')
dead_time_histogram('Выезд_из_технологической_секции', 'Номер плавки')

# Okaaay. What about the brigade number?
real_time_histogram('Поступление_в_технологическую_секцию', 'Номер_бригады')
dead_time_histogram('Поступление_в_технологическую_секцию', 'Номер_бригады')
real_time_histogram('Выезд_из_технологической_секции', 'Номер_бригады')
dead_time_histogram('Выезд_из_технологической_секции', 'Номер_бригады')

# Ok. Seems not human factor. Not correlated.

In [ ]:
# Not necessary. Just for show the way of thinking.
# Check the lengths.
def plot_hist(parameter):
    plt.hist(main_data[parameter])
    plt.xlabel(parameter)
    plt.show()
    plt.clf()

for i in range(1,6):
    plot_hist('Длина_'+str(i))

# Check the deltas.
limits = [-1000,8000]
bins = 1000
for i in range(1,5):
    deltas = main_data['Длина_'+str(i+1)].values - main_data['Длина_'+str(i)].values
    plt.hist(deltas, bins = 200, range = (0,8000))
    plt.title('Длина_'+str(i+1)+' - Длина_'+str(i))
    plt.xlabel('Delta(Length)')
    plt.ylabel('N')
    plt.show()
    plt.clf()
